In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
!pip install xgboost


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold, RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
#Models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score



In [5]:
df = pd.read_csv('C:/Maestría Inteligencia artificial aplicada/Trimestre 7 (Proyecto integrador y TD)/Semana 6/Relojes/new_df_afterPCA_relojes.csv')

In [6]:
df.head(5)

,PC1,PC2,PC3,PC4,PC5,PC6,ESTADO_SUCURSAL_BAJA CALIFORNIA NORTE,ESTADO_SUCURSAL_BAJA CALIFORNIA SUR,ESTADO_SUCURSAL_CAMPECHE,ESTADO_SUCURSAL_CHIAPAS,...,SUCURSAL_0,SUCURSAL_1,SUCURSAL_2,SUCURSAL_3,SUCURSAL_4,SUCURSAL_5,SUCURSAL_6,SUCURSAL_7,SUCURSAL_8,DIAS_ALMONEDA
0,-3.102492,-0.618919,-0.145880,-1.046442,0.404695,-0.266622,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,941
1,-0.796030,-0.612233,-0.431807,-1.584384,0.470162,0.210791,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,862
2,-0.517033,-1.420369,-0.350544,-0.148040,0.219067,0.086779,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,1,787
3,2.456877,0.028970,-0.361319,0.246168,-0.172364,-0.135953,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,783
4,4.601417,1.142606,-0.323991,0.801250,-0.447801,-0.373943,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,767


In [7]:
df.shape

(33925, 65)

In [8]:
df = df.dropna()

In [9]:
df.shape

(33925, 65)

Comentarios: A continuación vamos a particionar el dataframe en los datos que van a entrenar el modelo, validar y el testing. Como primer paso dejamos la variable objetivo a parte del dataframe de variables predictoras.

In [10]:
X = df.drop(df[['DIAS_ALMONEDA']],axis=1)
X

,PC1,PC2,PC3,PC4,PC5,PC6,ESTADO_SUCURSAL_BAJA CALIFORNIA NORTE,ESTADO_SUCURSAL_BAJA CALIFORNIA SUR,ESTADO_SUCURSAL_CAMPECHE,ESTADO_SUCURSAL_CHIAPAS,...,TIPO_PRENDA_reloj,SUCURSAL_0,SUCURSAL_1,SUCURSAL_2,SUCURSAL_3,SUCURSAL_4,SUCURSAL_5,SUCURSAL_6,SUCURSAL_7,SUCURSAL_8
0,-3.102492,-0.618919,-0.145880,-1.046442,0.404695,-0.266622,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,1
1,-0.796030,-0.612233,-0.431807,-1.584384,0.470162,0.210791,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,1,0
2,-0.517033,-1.420369,-0.350544,-0.148040,0.219067,0.086779,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,1,1
3,2.456877,0.028970,-0.361319,0.246168,-0.172364,-0.135953,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,0,1,0,0
4,4.601417,1.142606,-0.323991,0.801250,-0.447801,-0.373943,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33920,-0.783741,-2.993111,-0.568567,-0.489683,-1.875452,-0.640517,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,1,0,0,0
33921,-1.782705,-2.825057,-0.438366,-0.337646,-1.473801,-0.683229,0.0,0.0,0.0,0.0,...,1.0,0,0,0,0,0,1,0,0,0
33922,-3.202137,-2.451999,-0.408832,0.366651,-3.237874,6.180111,0.0,0.0,1.0,0.0,...,0.0,0,1,1,0,0,1,0,0,0
33923,2.712125,-1.150801,-0.499245,0.518431,-0.000846,-0.000287,0.0,0.0,0.0,0.0,...,1.0,0,1,1,1,1,0,1,1,1


In [11]:
y =df['DIAS_ALMONEDA'].copy()
y

0         941
1         862
2         787
3         783
4         767
         ... 
33920       9
33921       9
33922    2539
33923     698
33924     361
Name: DIAS_ALMONEDA, Length: 33925, dtype: int64

A continuación vamos a realizar una partición 70-15-15 en entrenamiento, validación y prueba respectivamente.

In [12]:
Xtrain, xtemp, ytrain, ytemp = train_test_split(X, y, test_size =.30)
Xval,Xtest, yval, ytest = train_test_split(xtemp, ytemp, test_size= 0.5)

In [13]:
print(Xtrain.shape, ytrain.shape)
print(Xval.shape, yval.shape)
print(Xtest.shape, ytest.shape)

(23747, 64) (23747,)
(5089, 64) (5089,)
(5089, 64) (5089,)


Comentarios. Como primer propuesta vamos a entrenar y evaluar un modelo de regresión lineal y podremos revisar si sus métricas son adecuadas

Vamos a evaluar el modelo con las métricas,raíz del error cuadrático medio (`RMSE`) y la métrica `R` cuadrado ($R^2$).

In [14]:
lr_model = LinearRegression()
lr_model.fit(Xtrain,ytrain)
y_prediction = lr_model.predict(Xval)

In [15]:
mse = mean_squared_error(yval,y_prediction)
rmse = np.sqrt(mse)
r2 = r2_score(yval,y_prediction)
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'R2: {r2:.4f}')

MSE: 37775.9678
RMSE: 194.3604
R2: 0.2641


Vamos a utilizar validación-cruzada, por lo que vamos a concatenar los conjuntos de entrenamiento y validación para utilizarlo como entrenamiento

In [16]:
Xtrainval = pd.concat([Xtrain,Xval],axis =0)
ytrainval = pd.concat([ytrain,yval],axis=0)

In [17]:
print(Xtrainval.shape, ytrainval.shape)

(28836, 64) (28836,)


Posterior a esto, vamos a proponer una función llamada "mis modelos" incluyendo los modelos que vamos a comparar y vamos a ajustar los parámetros para buscar el mejor modelo

In [23]:
def mis_modelos():
    modelos, nombres = list(), list()

    #ElasticNet
    modelos.append(ElasticNet(alpha=0.7, l1_ratio=0.5, tol=0.01,max_iter=1500,selection='random'))
    nombres.append('ElasticNet')

    #Árbol de Decisiones
    modelos.append(DecisionTreeRegressor(max_depth=15,max_features=15, min_samples_split=3))
    nombres.append('Dtree')

    #Random Forest
    modelos.append(RandomForestRegressor(n_estimators=700, max_features=25, max_depth=60,min_samples_split=0.5,max_samples=1.0,ccp_alpha=0.05))
    nombres.append('RandomF')

    #XGBoosting
    modelos.append(XGBRegressor(booster='gblinear',n_estimators=100, max_depth=60,learning_rate=0.2,subsample=1.0,objective= 'reg:squarederror'))
    nombres.append('XGBoost')

    #Red neuronal de Perceptrón Muliticapa:
    modelos.append(MLPRegressor(hidden_layer_sizes=(40,),activation='relu',max_iter=15,alpha=0.1,learning_rate='constant'))
    nombres.append('MLP')
    
    #Máquina de Vectores de Soporte:
    #modelos.append(SVR(kernel='rbf',C=1.0,gamma='scale',epsilon=0.1))#random_state=27))
    #nombres.append('SVM')

    return modelos, nombres

#Posteriormente vamos a entrenar cada uno de los modelos y vamos a desplegar las métricas de Train y val

modelos, nombres = mis_modelos()
resultados =list()

micv = RepeatedKFold(n_splits=10,n_repeats=2)

for modelo, nombre in zip(modelos,nombres):
    pipeline = Pipeline(steps=[('m',modelo)])


    #Se lleva a cabo el entrenamiento
    scores = cross_validate(pipeline,
                            Xtrainval,
                            ytrainval,
                            scoring={'r2':'r2', 'rmse': 'neg_root_mean_squared_error'},
                            cv=micv,
                            return_train_score=True,
                            n_jobs=1)
    #Se guarda el resultado de cada modelo
    resultados.append(scores)

    #Desplegamos los valores de las métricas para verificar si no hay subentrenamiento o sobreentrenamiento
    print(f">> {nombre}")
    print(f"\tTrain R2: {np.mean(scores['train_r2']):.3f} ± {np.std(scores['train_r2']):.3f}")
    print(f"\tVal   R2: {np.mean(scores['test_r2']):.3f} ± {np.std(scores['test_r2']):.3f}")
    print(f"\tTrain RMSE: {-np.mean(scores['train_rmse']):.3f} ± {np.std(scores['train_rmse']):.3f}")
    print(f"\tVal RMSE: {-np.mean(scores['test_rmse']):.3f} ± {np.std(scores['test_rmse']):.3f}")


>> ElasticNet
	Train R2: 0.197 ± 0.002
	Val   R2: 0.196 ± 0.013
	Train RMSE: 191.598 ± 1.016
	Val RMSE: 191.470 ± 9.776
>> Dtree
	Train R2: 0.735 ± 0.024
	Val   R2: 0.359 ± 0.064
	Train RMSE: 109.903 ± 5.251
	Val RMSE: 170.619 ± 12.554
>> RandomF
	Train R2: 0.271 ± 0.003
	Val   R2: 0.264 ± 0.014
	Train RMSE: 182.573 ± 0.640
	Val RMSE: 183.241 ± 6.603


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:55:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:55:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:55:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:55:54] WARNING: C:\ac

>> XGBoost
	Train R2: 0.248 ± 0.002
	Val   R2: 0.243 ± 0.021
	Train RMSE: 185.351 ± 0.870
	Val RMSE: 185.798 ± 7.858


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't 

>> MLP
	Train R2: 0.248 ± 0.009
	Val   R2: 0.245 ± 0.014
	Train RMSE: 185.446 ± 1.529
	Val RMSE: 185.473 ± 10.600


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


Podemos observar que la regresión lineal obtuvo una R^2 muy baja, de aproximadamente .26, muy por debajo de 1.  Con el modelo de Random Forest una R^2 de 0.29.

Posterior al Randomized Search que se realizó y el que tardó 2,288 minutos se obtuvieron los mejores parámetros. El ejercicio se realizó con los modelos XGboost, Random Forest y Decision Trees

XGBoost

In [21]:
param_dist_final_tune_xgboost = {
    'n_estimators': [10,20,30,100],
    'max_depth':[10,20,30,40,50,60],
    'learning_rate': [0.001,0.01,0.1,0.05,0.2,1.0],
    'subsample':[0.1,0.5,1.0,1.5,2.0],
    'booster':['gbtree','gblinear','dart']
}
xgb_tuned = XGBRegressor(objective='reg:squarederror',random_state=27)

cv = RepeatedKFold(n_splits=3,n_repeats=1,random_state=27)

random_search_final = RandomizedSearchCV(
    estimator=xgb_tuned,
    param_distributions=param_dist_final_tune_xgboost,
    n_iter=5,
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=1,
    n_jobs=1,
    random_state=27
)

print(" Resultados")
random_search_final.fit(Xtrain, ytrain)

                                                                                                    # Resultados
print("\ Mejor modelo ajustado:")
print(f"R2 validación cruzada: {random_search_final.best_score_:.4f}")
print("Mejores hiperparámetros:")
print(random_search_final.best_params_)

<>:27: SyntaxWarning: invalid escape sequence '\ '
<>:27: SyntaxWarning: invalid escape sequence '\ '
C:\Users\HP\AppData\Local\Temp\ipykernel_5008\1181478867.py:27: SyntaxWarning: invalid escape sequence '\ '
  print("\ Mejor modelo ajustado:")


 Resultados
Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:37:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:37:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:37:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:37:56] WARNING: C:\ac

\ Mejor modelo ajustado:
R2 validación cruzada: -33779.0898
Mejores hiperparámetros:
{'subsample': 1.0, 'n_estimators': 100, 'max_depth': 60, 'learning_rate': 0.2, 'booster': 'gblinear'}


Random Forest

In [22]:
param_dist_final_tune_RF = {
    'n_estimators': [10,20,30,100,200,300,400,700],
    'max_depth':[10,12,20,30,40,50,60],
    'min_samples_split':[0.1,0.5,1.0],
    'ccp_alpha': [0.001,0.1,0.05,.2,1.0],
    'max_samples':[0.1,0.5,1.0,0.2],
    'max_features':[10,14,20,25,30],
}
rf_tuned = RandomForestRegressor(criterion='squared_error',random_state=27)

cv = RepeatedKFold(n_splits=3,n_repeats=2,random_state=27)

random_search_final = RandomizedSearchCV(
    estimator=rf_tuned,
    param_distributions=param_dist_final_tune_RF,
    n_iter=5,
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=1,
    n_jobs=1
)
print("Resultados")
random_search_final.fit(Xtrain, ytrain)

                                                                                                    # Resultados
print("\ Mejor modelo ajustado:")
print(f"R2 validación cruzada: {random_search_final.best_score_:.4f}")
print("Mejores hiperparámetros:")
print(random_search_final.best_params_)

<>:26: SyntaxWarning: invalid escape sequence '\ '
<>:26: SyntaxWarning: invalid escape sequence '\ '
C:\Users\HP\AppData\Local\Temp\ipykernel_5008\4289566782.py:26: SyntaxWarning: invalid escape sequence '\ '
  print("\ Mejor modelo ajustado:")


Resultados
Fitting 6 folds for each of 5 candidates, totalling 30 fits
\ Mejor modelo ajustado:
R2 validación cruzada: -32986.6011
Mejores hiperparámetros:
{'n_estimators': 700, 'min_samples_split': 0.5, 'max_samples': 1.0, 'max_features': 25, 'max_depth': 60, 'ccp_alpha': 0.05}


Decision Trees

In [29]:
param_dist_final_tune_DT = {
    'max_depth':[10,12,20,30,40,50,60],
    'min_samples_split':[0.1,0.5,1.0],
    'max_leaf_nodes': [1,2,3,4],
    'ccp_alpha': [0.01,0.001,0.1,0.5,0.4],
    'max_features':[5,10,15,17],
    'criterion': ['squared_error','poisson','absolute_error']
}
rf_tuned = DecisionTreeRegressor()

cv = RepeatedKFold(n_splits=3,n_repeats=3)

random_search_final = RandomizedSearchCV(
    estimator=rf_tuned,
    param_distributions=param_dist_final_tune_DT,
    n_iter=5,
    scoring='r2',
    cv=cv,
    verbose=1,
    n_jobs=1
)
print("Resultados")
random_search_final.fit(Xtrain, ytrain)

                                                                                                    # Resultados
print("\ Mejor modelo ajustado:")
print(f"R2 validación cruzada: {random_search_final.best_score_:.4f}")
print("Mejores hiperparámetros:")
print(random_search_final.best_params_)

<>:26: SyntaxWarning: invalid escape sequence '\ '
<>:26: SyntaxWarning: invalid escape sequence '\ '
C:\Users\HP\AppData\Local\Temp\ipykernel_5008\2649216663.py:26: SyntaxWarning: invalid escape sequence '\ '
  print("\ Mejor modelo ajustado:")


Resultados
Fitting 9 folds for each of 5 candidates, totalling 45 fits
\ Mejor modelo ajustado:
R2 validación cruzada: 0.0755
Mejores hiperparámetros:
{'min_samples_split': 1.0, 'max_leaf_nodes': 4, 'max_features': 10, 'max_depth': 20, 'criterion': 'squared_error', 'ccp_alpha': 0.4}
